<a href="https://colab.research.google.com/github/fongbubble/UoB_EFIMM0142_Group12_NBA/blob/main/CEC_NBA_w_financial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\text{Supanu Tanatammatid 2546980}$$

## Package

In [1]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils

In [2]:
from pulp import GLPK
import pandas as pd
import numpy as np
import math
from tabulate import tabulate
from statistics import mean

In [3]:
url = "https://raw.githubusercontent.com/fongbubble/EFIMM0142_NBA_CEC_DEA/main/nba_updated_withrest1.csv"
data = pd.read_csv(url)
data = data.rename(columns={
    "Head Coach's Experience": "Head_Coachs_Experience",
    "Head Coach's Ability": "Head_Coachs_Ability",
    "No. of Games with Rest ": "No_of_Games_with_Rest"
})

### CCR

In [4]:
weights_matrix = np.empty((16, 5))
objective_values = np.empty(16)
for k in range(5):
    for i in range(16):
        model = pulp.LpProblem("NBA", pulp.LpMaximize)  # Create an LP maximization problem

        # Create decision variables
        u1 = pulp.LpVariable("u1", lowBound=0, upBound=None, cat='Continuous')  # Games Won
        u2 = pulp.LpVariable("u2", lowBound=0, upBound=None, cat='Continuous')  # Playoff -Depth
        u3 = pulp.LpVariable("u3", lowBound=0, upBound=None, cat='Continuous')  # Average Margin of Victory
        v1 = pulp.LpVariable("v1", lowBound=0, upBound=None, cat='Continuous')  # AFTbudget
        v2 = pulp.LpVariable("v2", lowBound=0, upBound=None, cat='Continuous')  # ABTBudget

        # Objective function
        model += (
            u1 * data["Games Won"].iloc[i] +
            u2 * data["Playoff -Depth"].iloc[i] +
            u3 * data["Average Margin of Victory"].iloc[i]
        )

        # First constraint (equality)
        model += (
            v1 * data["AFTbudget"].iloc[i] +
            v2 * data["ABTBudget"].iloc[i] == 1
        )

        # Additional constraints
        for j in range(16):
            model += (
                u1 * data["Games Won"].iloc[j] +
                u2 * data["Playoff -Depth"].iloc[j] +
                u3 * data["Average Margin of Victory"].iloc[j] -
                v1 * data["AFTbudget"].iloc[j] -
                v2 * data["ABTBudget"].iloc[j] <= 0
            )

        # Solve the problem
        status = model.solve(pulp.GLPK(msg=True, options=['--ranges', 'sensitivity_w_financial.txt']))
        list_weights = []
        for v in model.variables():
            list_weights.append(v.varValue)

        weights_matrix[i, 0] = list_weights[0]  # u1 -> Column 0 (Scenario_1)
        weights_matrix[i, 1] = list_weights[1]  # u2 -> Column 1 (Scenario_2)
        weights_matrix[i, 2] = list_weights[2]  # u3 -> Column 2 (Scenario_3)
        weights_matrix[i, 3] = list_weights[3]  # v1 -> Column 3 (Scenario_4)
        weights_matrix[i, 4] = list_weights[4]  # v2 -> Column 4 (Scenario_5)
        objective_value = pulp.value(model.objective)
        objective_values[i] = objective_value

team_names = data['Team'].tolist()
scenario_names = [f'Scenario_{k+1}' for k in range(5)]
weights_df = pd.DataFrame(weights_matrix, index=team_names, columns=scenario_names)
objective_values_array = objective_values.reshape(16, 1)


### Second Objective

In [5]:
CEC_weights_matrix = np.empty((16, 5))
for k in range(5):
    for i in range(16):
        Second_Objective_model = pulp.LpProblem("NBA_CEC", pulp.LpMaximize)

        # Define decision variables
        u1 = pulp.LpVariable("u1", lowBound=0, upBound=None, cat='Continuous')  # Games Won
        u2 = pulp.LpVariable("u2", lowBound=0, upBound=None, cat='Continuous')  # Playoff -Depth
        u3 = pulp.LpVariable("u3", lowBound=0, upBound=None, cat='Continuous')  # Average Margin of Victory
        v1 = pulp.LpVariable("v1", lowBound=0, upBound=None, cat='Continuous')  # AFTbudget
        v2 = pulp.LpVariable("v2", lowBound=0, upBound=None, cat='Continuous')  # ABTBudget

        # Define the objective function
        Second_Objective_model += (
            u1 * data["Games Won"].sum() +
            u2 * data["Playoff -Depth"].sum() +
            u3 * data["Average Margin of Victory"].sum()
            - (v1 * data["AFTbudget"].sum() +
            v2 * data["ABTBudget"].sum())
        )

        # Define the equality constraint
        Second_Objective_model += (
            v1 * data["AFTbudget"].iloc[i] +
            v2 * data["ABTBudget"].iloc[i] == 1
        )

        # Define the inequalities for each team
        for j in range(16):
            Second_Objective_model += (
                u1 * data["Games Won"].iloc[j] +
                u2 * data["Playoff -Depth"].iloc[j] +
                u3 * data["Average Margin of Victory"].iloc[j]
                - (v1 * data["AFTbudget"].iloc[j] +
                   v2 * data["ABTBudget"].iloc[j]) <= 0
            )

        # Additional constraint using the list of objective values
        Second_Objective_model += (
            u1 * data["Games Won"].iloc[i] +
            u2 * data["Playoff -Depth"].iloc[i] +
            u3 * data["Average Margin of Victory"].iloc[i]
            - (objective_values_array[i, 0] * (
                v1 * data["AFTbudget"].iloc[i] +
                v2 * data["ABTBudget"].iloc[i])) == 0
        )

        # Solve the model
        status = Second_Objective_model.solve()
        print("Model Status:{}".format(pulp.LpStatus[Second_Objective_model.status]))
        new_list_weights = []
        for v in Second_Objective_model.variables():
            v.varValue = max(0, v.varValue)
            new_list_weights.append(v.varValue)
            print(v.name, "=", v.varValue)

        CEC_weights_matrix[i, 0] = new_list_weights[0]  # u1 -> Column 0 (Scenario_1)
        CEC_weights_matrix[i, 1] = new_list_weights[1]  # u2 -> Column 1 (Scenario_2)
        CEC_weights_matrix[i, 2] = new_list_weights[2]  # u3 -> Column 2 (Scenario_3)
        CEC_weights_matrix[i, 3] = new_list_weights[3]  # v1 -> Column 3 (Scenario_4)
        CEC_weights_matrix[i, 4] = new_list_weights[4]  # v2 -> Column 4 (Scenario_5)
        print("\nConstraint")
        for name, c in Second_Objective_model.constraints.items():
            print(name, ":", c)

team_names = data['Team'].tolist()
scenario_names = [f'Scenario_{k+1}' for k in range(5)]
CEC_weights_df = pd.DataFrame(CEC_weights_matrix, index=team_names, columns=scenario_names)
print("Final CEC Weights Matrix with Team Names and Scenario Labels:")
print(CEC_weights_df)

Model Status:Optimal
u1 = 0.015625
u2 = 0
u3 = 0
v1 = 0.023530088
v2 = 0.10333657

Constraint
_C1 : 31.3*v1 + 2.55*v2 = 1.0
_C2 : 64*u1 + 5*u2 + 11.3*u3 - 31.3*v1 - 2.55*v2 <= -0.0
_C3 : 57*u1 + 2*u2 + 5.3*u3 - 30.4*v1 - 2.61*v2 <= -0.0
_C4 : 57*u1 + 2*u2 + 7.4*u3 - 14.0*v1 - 5.64*v2 <= -0.0
_C5 : 56*u1 + 3*u2 + 6.5*u3 - 23.8*v1 - 3.51*v2 <= -0.0
_C6 : 51*u1 + u2 + 3.3*u3 - 31.2*v1 - 4.01*v2 <= -0.0
_C7 : 50*u1 + 4*u2 + 2.2*u3 - 23.3*v1 - 4.27*v2 <= -0.0
_C8 : 50*u1 + 2*u2 + 4.6*u3 - 22.0*v1 - 3.98*v2 <= -0.0
_C9 : 49*u1 + u2 + 2.6*u3 - 29.5*v1 - 3.46*v2 <= -0.0
_C10 : 49*u1 + u2 + 4.4*u3 - 23.8*v1 - 3.53*v2 <= -0.0
_C11 : 48*u1 + u2 + 2.2*u3 - 28.5*v1 - 3.75*v2 <= -0.0
_C12 : 48*u1 + 2*u2 + 2.4*u3 - 22.1*v1 - 3.58*v2 <= -0.0
_C13 : 47*u1 + 3*u2 + 3.0*u3 - 15.4*v1 - 3.6*v2 <= -0.0
_C14 : 47*u1 + u2 + 0.6*u3 - 24.4*v1 - 4.61*v2 <= -0.0
_C15 : 47*u1 + u2 + 2.0*u3 - 10.1*v1 - 8.05*v2 <= -0.0
_C16 : 47*u1 + u2 + 3.0*u3 - 19.2*v1 - 4.42*v2 <= -0.0
_C17 : 46*u1 + u2 + 1.8*u3 - 26.3*v1 - 3.93

### Cross-efficiency matrix

In [6]:
CEC_efficiency_table = np.empty((16, 16))
CEC_average_list = []
for team in range(16):
    row_values = []
    for scenario in range(16):
        numerator = (
            (CEC_weights_matrix[scenario, 0] * data['Games Won'].iloc[team]) +
            (CEC_weights_matrix[scenario, 1] * data['Playoff -Depth'].iloc[team]) +
            (CEC_weights_matrix[scenario, 2] * data['Average Margin of Victory'].iloc[team])
        )
        denominator = (
            (CEC_weights_matrix[scenario, 3] * data['AFTbudget'].iloc[team]) +
            (CEC_weights_matrix[scenario, 4] * data['ABTBudget'].iloc[team])
        )
        if denominator == 0:
            denominator = 1e-10
        value = numerator / denominator
        CEC_efficiency_table[team, scenario] = value
        row_values.append(value)
    average = np.mean(row_values)
    CEC_average_list.append(average)

In [7]:
CEC_efficiency_df = pd.DataFrame(CEC_efficiency_table, index=team_names, columns=[name + "_weight" for name in team_names])
print("Efficiency Table with Team Names:")
print(CEC_efficiency_df.to_string(formatters={col: '{:.2f}'.format for col in CEC_efficiency_df.columns}))
csv_filename = "2nd_efficiency_table_w_financial.csv"
CEC_efficiency_df.to_csv(csv_filename, index=True)

Efficiency Table with Team Names:
                       Boston Celtics_weight Denver Nuggets_weight Oklahoma City Thunder_weight Minnesota Timberwolves_weight Los Angeles Clippers_weight Dallas Mavericks_weight New York Knicks_weight Milwaukee Bucks_weight New Orleans Pelicans_weight Phoenix Suns_weight Cleveland Cavaliers_weight Indiana Pacers_weight Los Angeles Lakers_weight Orlando Magic_weight Philadelphia 76ers_weight Miami Heat_weight
Boston Celtics                          1.00                  1.00                         1.00                          1.00                        1.00                    1.00                   1.00                   1.00                        1.00                1.00                       1.00                  1.00                      1.00                 0.57                      1.00              1.00
Denver Nuggets                          0.90                  0.90                         0.89                          0.90                 

### Arithmetic mean for average cross-efficiency score

In [8]:
CEC_team_efficiency = list(zip(data['Team'], CEC_average_list))
sorted_CEC_team_efficiency = sorted(CEC_team_efficiency, key=lambda x: x[1], reverse=True)
for team, efficiency in sorted_CEC_team_efficiency:
    print(f"Cross-efficiency score for {team} = {round(efficiency, 5)}")

Cross-efficiency score for Indiana Pacers = 0.98741
Cross-efficiency score for Boston Celtics = 0.97303
Cross-efficiency score for Oklahoma City Thunder = 0.95645
Cross-efficiency score for Minnesota Timberwolves = 0.91458
Cross-efficiency score for Denver Nuggets = 0.84843
Cross-efficiency score for Cleveland Cavaliers = 0.8051
Cross-efficiency score for New York Knicks = 0.80492
Cross-efficiency score for Dallas Mavericks = 0.78477
Cross-efficiency score for New Orleans Pelicans = 0.77396
Cross-efficiency score for Philadelphia 76ers = 0.76413
Cross-efficiency score for Orlando Magic = 0.68325
Cross-efficiency score for Milwaukee Bucks = 0.67731
Cross-efficiency score for Phoenix Suns = 0.661
Cross-efficiency score for Los Angeles Lakers = 0.657
Cross-efficiency score for Miami Heat = 0.65609
Cross-efficiency score for Los Angeles Clippers = 0.64617


### Rescale matrix

In [9]:
# Step 1: Find the row with the maximum count of values close to 1.0
row_with_max_ones = 11

# Step 2: Rescale Matrix
Rescale_Matrix = np.zeros_like(CEC_efficiency_table)
for i in range(CEC_efficiency_table.shape[1]):
    Rescale_Matrix[:, i] = CEC_efficiency_table[:, i] / (CEC_efficiency_table[row_with_max_ones, i] + 1e-10)
Rescale_Matrix = np.round(Rescale_Matrix, 4)

### Cross-efficiency consensus score

In [10]:
# Step 3: Compute Geometric Mean for Each DMU (Rows)
geometric_mean = [np.prod(Rescale_Matrix[i, :]) ** (1 / Rescale_Matrix.shape[1]) for i in range(Rescale_Matrix.shape[0])]

# Step 4: Calculate Root Sum of Squared Deviations
root_sum_squared_list = []
for col in range(Rescale_Matrix.shape[1]):
    column = Rescale_Matrix[:, col]
    mean = geometric_mean[col]
    deviations = math.sqrt(sum((x - mean) ** 2 for x in column))
    root_sum_squared_list.append(deviations)

# Step 5: Calculate Evaluation Consensus Degree (ECD)
ECD = [1 / (1 + x) for x in root_sum_squared_list]

# Step 6: Calculate Consensus Efficiency Cross (CEC) Results
total_power = sum(ECD)
CEC_Results = []
for i in range(len(ECD)):
    row = Rescale_Matrix[i]
    product = np.prod(np.power(row + 1e-10, ECD[i]))
    result = np.power(product, 1 / total_power)
    CEC_Results.append(result)
CEC_Results = np.array(CEC_Results).round(decimals=5)

# Step 7: Rank All DMUs Based on Final Aggregated Efficiency Scores
df_results = pd.DataFrame({
    'Team': team_names,
    'Consensus Efficiency Score': CEC_Results
})
ECD_df = pd.DataFrame(ECD)
ECD_df.insert(0, 'Team', team_names)
ECD_df.to_csv('ECD_output_w_financial.csv', index=False)
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)

In [11]:
df_results = pd.DataFrame({
    'Team': team_names,
    'Consensus Efficiency Score': CEC_Results
})
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)
for index, row in df_sorted.iterrows():
    print(f"Cross-efficiency consensus score for {row['Team']} = {row['Consensus Efficiency Score']:.4f}")

Cross-efficiency consensus score for Indiana Pacers = 1.0000
Cross-efficiency consensus score for Boston Celtics = 0.9796
Cross-efficiency consensus score for Oklahoma City Thunder = 0.9649
Cross-efficiency consensus score for Minnesota Timberwolves = 0.9165
Cross-efficiency consensus score for Dallas Mavericks = 0.8640
Cross-efficiency consensus score for Denver Nuggets = 0.8162
Cross-efficiency consensus score for New York Knicks = 0.7773
Cross-efficiency consensus score for Cleveland Cavaliers = 0.7773
Cross-efficiency consensus score for New Orleans Pelicans = 0.7300
Cross-efficiency consensus score for Philadelphia 76ers = 0.7271
Cross-efficiency consensus score for Orlando Magic = 0.7069
Cross-efficiency consensus score for Milwaukee Bucks = 0.6710
Cross-efficiency consensus score for Los Angeles Lakers = 0.6643
Cross-efficiency consensus score for Phoenix Suns = 0.6643
Cross-efficiency consensus score for Miami Heat = 0.6631
Cross-efficiency consensus score for Los Angeles Clipp

### Results comparison

In [12]:
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)
CEC_average_df = pd.DataFrame(CEC_average_list, columns=['Average Cross-efficiency'], index=team_names)
merged_df = pd.merge(df_sorted, CEC_average_df, left_on='Team', right_index=True)
merged_df = merged_df[['Team', 'Average Cross-efficiency', 'Consensus Efficiency Score']]
merged_df['Average Cross-efficiency'] = merged_df['Average Cross-efficiency'].round(4)
merged_df['Consensus Efficiency Score'] = merged_df['Consensus Efficiency Score'].round(4)
beautiful_table = tabulate(merged_df, headers='keys', tablefmt='pretty', showindex=False)
print(beautiful_table)

+------------------------+--------------------------+----------------------------+
|          Team          | Average Cross-efficiency | Consensus Efficiency Score |
+------------------------+--------------------------+----------------------------+
|     Indiana Pacers     |          0.9874          |            1.0             |
|     Boston Celtics     |          0.973           |           0.9796           |
| Oklahoma City Thunder  |          0.9564          |           0.9649           |
| Minnesota Timberwolves |          0.9146          |           0.9165           |
|    Dallas Mavericks    |          0.7848          |           0.864            |
|     Denver Nuggets     |          0.8484          |           0.8162           |
|    New York Knicks     |          0.8049          |           0.7773           |
|  Cleveland Cavaliers   |          0.8051          |           0.7773           |
|  New Orleans Pelicans  |          0.774           |            0.73            |
|   